![Roboflow Notebooks banner](https://camo.githubusercontent.com/aec53c2b5fb6ed43d202a0ab622b58ba68a89d654fbe3abab0c0cc8bd1ff424e/68747470733a2f2f696b2e696d6167656b69742e696f2f726f626f666c6f772f6e6f7465626f6f6b732f74656d706c6174652f62616e6e657274657374322d322e706e673f696b2d73646b2d76657273696f6e3d6a6176617363726970742d312e342e33267570646174656441743d31363732393332373130313934)

# Image Classification with DINOv2

DINOv2, released by Meta Research in April 2023, implements a self-supervised method of training computer vision models.

DINOv2 was trained using 140 million images without labels. The embeddings generated by DINOv2 can be used for classification, image retrieval, segmentation, and depth estimation. With that said, Meta Research did not release heads for segmentation and depth estimation.
In this guide, we are going to build an image classifier using embeddings from DINOv2. To do so, we will:

1. Load a folder of images
2. Compute embeddings for each image
3. Save all the embeddings in a file and vector store
4. Train an SVM classifier to classify images

By the end of this notebook, we'll have a classifier trained on our dataset.

Without further ado, let's begin!

## Import Packages

First, let's import the packages we will need for this project.

In [1]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import os
#import cv2
import json
import glob
from tqdm.notebook import tqdm

In [2]:
cwd = os.getcwd()
cwd

'/workspaces/gc_quant_trading_research'

Load folder containing the trading images

In [3]:
cwd = os.getcwd()

ROOT_DIR = os.path.join(cwd)

labels = {}
# remove the folder Now from the for loop

for folder in os.listdir(ROOT_DIR):
  if folder == 'test':
    continue 
  else:
    try:
      print(folder)
      for file in os.listdir(os.path.join(ROOT_DIR, folder)):
          if file.endswith(".png"):
              full_name = os.path.join(ROOT_DIR, folder, file)
              labels[full_name] = folder
    except:
      pass

files = labels.keys()

2D_embedding_viz.ipynb
all_embeddings.json
bear
# %% [markdown]
now
Dinov2_classification_gc.ipynb
classification_app.py
2d_scatter_plot_4candles.html
2d_scatter_plot.html
bull
README.md
models
ml_experiment_config.toml
dockerfile
notebooks
.git
data
requirements.txt


In [4]:
list(files)

['/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-17 at 20.48.16.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-17 at 20.49.06.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-17 at 20.40.14.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-10 at 14.38.50.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-17 at 20.37.33.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-10 at 14.47.46.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-10 at 14.51.20.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-10 at 14.48.14.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-17 at 20.44.38.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-10 at 14.43.38.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-17 at 20.43.04.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-1

In [5]:
# prompt: get data from dictionary files

values = [labels[key] for key in files]

## Load the Model and Compute Embeddings

To train our classifier, we need:

1. The embeddings associated with each image in our dataset, and;
2. The labels associated with each image.

To calculate embeddings, we'll use DINOv2. Below, we load the smallest DINOv2 weights and define functions that will load and compute embeddings for every image in a specified list.

We store all of our vectors in a dictionary that is saved to disk so we can reference them again if needed. Note that in production environments one may opt for using another data structure such as a vector embedding database (i.e. faiss) for storing embeddings.

In [18]:
MODEL = "dinov2_vits14"
EMBEDDING_SIZE = 384 # 768 for vitb14, 384 for vits14

dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", MODEL)

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

dinov2_vits14.to(device)

transform_image = T.Compose([T.ToTensor(),
                             T.Resize((70, 210)),
                             #T.CenterCrop(224),
                             T.Normalize([0.5], [0.5])])

Using cache found in /home/codespace/.cache/torch/hub/facebookresearch_dinov2_main


In [19]:
def load_image(img: str) -> torch.Tensor:
    """
    Load an image and return a tensor that can be used as an input to DINOv2.
    """
    img = Image.open(img)

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img

def compute_embeddings(files: list) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    all_embeddings = {}

    with torch.no_grad():
      for i, file in enumerate(files):
        embeddings = dinov2_vits14(load_image(file).to(device))

        all_embeddings[file] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()

    with open("all_embeddings.json", "w") as f:
        f.write(json.dumps(all_embeddings))

    return all_embeddings

## Compute Embeddings

The code below computes the embeddings for all the images in our dataset. This step will take a few minutes for the MIT Indoor Scene Recognition dataset. There are over 10,000 images in the training set that we need to pass through DINOv2.

In [20]:
embeddings = compute_embeddings(files)

In [21]:
#get 1st dictionary key value
key = list(embeddings.keys())[0]
embeddings[key]

[[-5.4831743240356445,
  1.8402979373931885,
  -2.580328941345215,
  -3.108518123626709,
  -1.7923815250396729,
  -0.1158885806798935,
  0.20237015187740326,
  -0.14818476140499115,
  2.854886054992676,
  2.9640865325927734,
  0.5049887895584106,
  0.6332003474235535,
  -2.9214816093444824,
  -2.1240949630737305,
  2.8888816833496094,
  -2.9483284950256348,
  -0.721474826335907,
  -6.388484477996826,
  -1.2936780452728271,
  -2.4729442596435547,
  4.254789352416992,
  1.237850308418274,
  1.9444843530654907,
  0.020482327789068222,
  1.6316792964935303,
  0.31837525963783264,
  1.5959244966506958,
  5.1202921867370605,
  3.481863498687744,
  -2.9009883403778076,
  1.123852252960205,
  2.906263589859009,
  2.381955146789551,
  -0.16986925899982452,
  -4.108577251434326,
  -0.7669607400894165,
  1.7991366386413574,
  -1.826431393623352,
  0.38951143622398376,
  3.0916924476623535,
  -4.6494622230529785,
  -0.7016718983650208,
  -5.171243667602539,
  2.5149338245391846,
  2.27685737609863

In [22]:
# Check the shape of embedding_list before reshaping
embedding_list = list(embeddings.values())
embedding_array = np.array(embedding_list)
print(f"Original shape: {embedding_array.shape}")
print(f"Total number of elements: {embedding_array.size}")

# Attempt to reshape
reshaped_array = embedding_array.reshape(-1, EMBEDDING_SIZE)
print(f"Reshaped array shape: {reshaped_array.shape}")

Original shape: (416, 1, 384)
Total number of elements: 159744
Reshaped array shape: (416, 384)


## Train a Classification Model

The embeddings we have computed can be used as an input in a classification model. For this guide, we will be using SVM, a linear classification model.

Below, we make lists of both all of the embeddings we have computed and their associated labels. We then fit our model using those lists.

In [23]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

y = [labels[file] for file in files]
embedding_list = list(embeddings.values())
embedding_arr = np.array(embedding_list).reshape(-1, EMBEDDING_SIZE)
N_COMPONENTS = 100
N_ESTIMATORS = 100
MAX_DEPTH = 100
R_STATE = 5

#fit a svm model
def fit_svm_pca(embedding_list, y):
    clf = svm.SVC(gamma='scale')
    # Convert embedding_list to a 2D array
    embedding_array = np.array(embedding_list).reshape(len(embedding_list), -1)
    
    # Apply PCA to reduce dimensions to x principal components
    pca = PCA(n_components=N_COMPONENTS, random_state=R_STATE)
    embedding_list = pca.fit_transform(embedding_array)
    
    clf.fit(embedding_list, y)
    return clf, pca

clf_svm_pca, pca = fit_svm_pca(embedding_list, y)


#fit a svm model with pca
def fit_svm(embedding_list, y):
    clf = svm.SVC(gamma='scale')
    clf.fit(np.array(embedding_list).reshape(-1, EMBEDDING_SIZE), y)
    return clf

clf_svm = fit_svm(embedding_list, y)


#fit a random forest model
def fit_rf(embedding_list, y):
    rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=R_STATE) #70
    rf.fit(np.array(embedding_list).reshape(-1, EMBEDDING_SIZE), y)
    return rf

clf_rf = fit_rf(embedding_list, y)
#save this rf model


#fit a random forest model with pca
def fit_rf_pca(embedding_list, y):
    rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=R_STATE) #70

    # Convert embedding_list to a 2D array
    embedding_array = np.array(embedding_list).reshape(len(embedding_list), -1)
    
    # Apply PCA to reduce dimensions to x principal components
    pca = PCA(n_components=N_COMPONENTS, random_state=R_STATE)
    embedding_list = pca.fit_transform(embedding_array)

    rf.fit(embedding_list, y)
    return rf, pca

clf_rf_pca, rf_pca = fit_rf_pca(embedding_list, y)




In [24]:
import joblib
save_model = "no"
if save_model == "yes":

    joblib.dump(clf_rf_pca, 'models/rf_pca/best_rf_model_oct1st.pkl')
    joblib.dump(rf_pca, 'models/rf_pca/best_pca_model.pkl')

## Classify an Image

We now have a classifier we can use to classify images!

Change the `input_file` value below to the path of a file in the `valid` or `test` directories in the image dataset with which we have been working.

Then, run the cell to classify the image.

In [25]:
#any file in the folder title Now
TEST_FOLDER = "bear"
input_files = glob.glob(f'test/{TEST_FOLDER}/*.png')

for file in glob.glob(f'test/bull/*.png'):
    input_files.append(file)


In [26]:
input_files

['test/bear/Screenshot 2024-10-17 at 20.50.53.png',
 'test/bear/Screenshot 2024-10-17 at 20.51.09.png',
 'test/bear/Screenshot 2024-10-17 at 20.51.31.png',
 'test/bear/Screenshot 2024-10-17 at 20.49.36.png',
 'test/bear/Screenshot 2024-10-17 at 20.51.02.png',
 'test/bear/Screenshot 2024-10-17 at 20.51.21.png',
 'test/bear/Screenshot 2024-10-17 at 20.50.45.png',
 'test/bear/Screenshot 2024-10-17 at 20.49.11.png',
 'test/bear/Screenshot 2024-10-17 at 20.51.39.png',
 'test/bear/Screenshot 2024-10-17 at 20.49.19.png',
 'test/bull/Screenshot 2024-10-17 at 20.17.47.png',
 'test/bull/Screenshot 2024-10-17 at 20.18.43.png',
 'test/bull/Screenshot 2024-10-17 at 20.17.41.png',
 'test/bull/Screenshot 2024-10-17 at 20.18.10.png',
 'test/bull/Screenshot 2024-10-17 at 20.18.03.png',
 'test/bull/Screenshot 2024-10-17 at 20.18.18.png',
 'test/bull/Screenshot 2024-10-17 at 20.32.57.png',
 'test/bull/Screenshot 2024-10-17 at 20.33.09.png',
 'test/bull/Screenshot 2024-10-17 at 20.32.50.png',
 'test/bull/

In [27]:
# Initialize an empty DataFrame
import pandas as pd

predictions_df = pd.DataFrame(columns=['image_file', 'svm_prediction', 'svm_pca_prediction', 'rf_prediction', 
                                       'rf_pca_prediction'])



for input_file in input_files:
    new_image = load_image(input_file)

    with torch.no_grad():
        new_embedding = dinov2_vits14(new_image.to(device))

        # Convert embedding to numpy array and reshape
        new_embedding_array = np.array(new_embedding[0].cpu()).reshape(1, -1)
        new_embedding_pca = pca.transform(new_embedding_array)
        
        # Generate with newly trained model the predictions
        svm_pca_prediction = clf_svm_pca.predict(new_embedding_pca)
        svm_prediction = clf_svm.predict(new_embedding_array)
        rf_prediction = clf_rf.predict(new_embedding_array)
        rf_pca_prediction = clf_rf_pca.predict(new_embedding_pca)



 

        # Add the predictions to the DataFrame using loc
        predictions_df.loc[len(predictions_df)] = [input_file, svm_prediction[0],svm_pca_prediction[0],
                                                    rf_prediction[0], rf_pca_prediction[0]]


In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Extract true labels from the file paths
predictions_df['true_label'] = predictions_df['image_file'].apply(lambda x: 'bull' if 'bull' in x else 'bear')


# Compute accuracy for SVM and RF predictions
svm_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['svm_prediction'])
svm_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['svm_pca_prediction'])
rf_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['rf_prediction'])
rf_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['rf_pca_prediction'])


print('ALL')
print('--------------------------------------------------')
print(f"SVM Accuracy: {round(svm_accuracy, 2)}")
print(f"RF Accuracy: {round(rf_accuracy,2)}")
print(f"SVM PCA Accuracy: {round(svm_pca_accuracy,2)}")
print(f"RF PCA Accuracy: {round(rf_pca_accuracy,2)}")
print('--------------------------------------------------')


predictions_df

ALL
--------------------------------------------------
SVM Accuracy: 0.52
RF Accuracy: 0.43
SVM PCA Accuracy: 0.52
RF PCA Accuracy: 0.48
--------------------------------------------------


,image_file,svm_prediction,svm_pca_prediction,rf_prediction,rf_pca_prediction,true_label
0,test/bear/Screenshot 2024-10-17 at 20.50.53.png,bear,bear,bear,bear,bear
1,test/bear/Screenshot 2024-10-17 at 20.51.09.png,bull,bull,bull,bull,bear
2,test/bear/Screenshot 2024-10-17 at 20.51.31.png,bear,bull,bull,bear,bear
3,test/bear/Screenshot 2024-10-17 at 20.49.36.png,bear,bull,bull,bear,bear
4,test/bear/Screenshot 2024-10-17 at 20.51.02.png,bear,bear,bear,bear,bear
5,test/bear/Screenshot 2024-10-17 at 20.51.21.png,bear,bull,bull,bull,bear
6,test/bear/Screenshot 2024-10-17 at 20.50.45.png,bull,bull,bull,bear,bear
7,test/bear/Screenshot 2024-10-17 at 20.49.11.png,bear,bear,bear,bear,bear
8,test/bear/Screenshot 2024-10-17 at 20.51.39.png,bear,bull,bear,bull,bear
9,test/bear/Screenshot 2024-10-17 at 20.49.19.png,bear,bear,bear,bear,bear
